In [126]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as soup
import lxml
from bs4 import NavigableString
import re

In [127]:
# Must gather rating data for all the games without data in the ratings_df in the exploratory analysis notebook.
# Must gather positive, neutral, and negative ratings numbers for both users and critics as well as the
# metascore and userscore

In [128]:
no_rating_df = pd.read_csv("../Resources/games_to_get_ratings.csv")

In [129]:
!which chromedriver

/usr/local/bin/chromedriver


In [160]:
browser = Browser("chrome", executable_path="/usr/local/bin/chromedriver", headless=True)

In [131]:
games_df = pd.read_csv("../Resources/vgsales.csv.zip")
list_of_games = games_df["Name"].values.tolist()

In [17]:
# Grabbing the Metascore

game_scores = {}
problem_games = []

for game in ["Super Smash Bros."]:
    browser.visit("https://www.metacritic.com/")
    # fill in my search term to the Metacritic Browser and us "\n" to press "enter" and initialize search
    browser.fill("search_term", game + "\n")
    html= browser.html
    my_soup = soup(html, "lxml")
    try:
        metascore = my_soup.find("a", class_="metascore_anchor").get_text()
        game_scores[game]= metascore
        print("SUCCESSFUL")
    except:
        problem_games.append(game)
        print("THIS IS A PROBLEM CHILD")

THIS IS A PROBLEM CHILD


In [132]:
my_df = pd.DataFrame(game_scores, columns=["Name","Metascore"])

In [133]:
## Navigating to the correct page via the search using splinter is too slow. I will rewrite the code to manually 
# write in the url. 
# I will need to interpolate the correct console name as well into the url. Metacritics url structure is:
# "https://www.metacritic.com/game/{console}/{game}"

In [134]:
# must change game names into the proper format to be interpolated into the URL.

game_conversions= {
    "PSV":"playstation-vita",
    "DC":"dreamcast",
    "WiiU":"wii-u",
    "GC":"gamecube",
    "N64":"nintendo-64",
    "XB":"xbox",
    "PSP":"psp",
    "PS4":"playstation-4",
    "PS": "playstation",
    "Wii":"wii",
    "PS3": "playstation-3",
    "PS2":"playstation-2",
    "GBA":"game-boy-advance",
    "DS":"ds",
    "XB":"xbox",
    "X360":"xbox-360"
}

def convert_to_name(platform_abb):
    if platform_abb in game_conversions:
        full_platform= game_conversions[platform_abb]
    else:
        full_platform="other"
    return full_platform
    

In [159]:
# This actually shows that the vast majority of our data belongs to the consoles listed on metacritic, these "legacy consoles"
# Get rid of those games which fall into the "other" category under full_platform_name
games_df["full_platform_name"]= games_df["Platform"].map(convert_to_name)
games_df = games_df.loc[games_df["full_platform_name"]!="other"]


In [115]:
# Grabbing the Metascore

game_scores = {}
problem_games = []


# zip the two 
zipped_list =list(zip(games_df["full_platform_name"].tolist(),games_df["Name"].tolist()))

for platform, name in zipped_list:
    edited_game_title= name.lower().replace(":","-").replace(" ","-").replace("\W|\s","-").replace("--","-")
    browser.visit(f"https://www.metacritic.com/game/{platform}/{edited_game_title}")
    # fill in my search term to the Metacritic Browser and us "\n" to press "enter" and initialize search
    
    html= browser.html
    my_soup = soup(html, "lxml")
    try:
        metascore = my_soup.find("a", class_="metascore_anchor").get_text()
        game_scores[game]= metascore
        print("SUCCESSFUL")
    except:
        problem_games.append(game)
        print("THIS IS A PROBLEM CHILD")

SUCCESSFUL
THIS IS A PROBLEM CHILD
SUCCESSFUL
SUCCESSFUL
THIS IS A PROBLEM CHILD
THIS IS A PROBLEM CHILD
THIS IS A PROBLEM CHILD
SUCCESSFUL
THIS IS A PROBLEM CHILD
THIS IS A PROBLEM CHILD
THIS IS A PROBLEM CHILD
SUCCESSFUL
THIS IS A PROBLEM CHILD
SUCCESSFUL
SUCCESSFUL
SUCCESSFUL
THIS IS A PROBLEM CHILD
SUCCESSFUL
THIS IS A PROBLEM CHILD
SUCCESSFUL
THIS IS A PROBLEM CHILD
THIS IS A PROBLEM CHILD
THIS IS A PROBLEM CHILD
SUCCESSFUL
SUCCESSFUL
THIS IS A PROBLEM CHILD
THIS IS A PROBLEM CHILD
SUCCESSFUL


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=86.0.4240.198)


In [21]:
# This worked pretty well, but I will alter the code further see if I can increase my scraping success to fail ratio

In [136]:
import re
mystring = "The Legend!of:Zelda; Breath of the wild"
x = re.compile("\W|\s")
mystring = re.sub(x,"-", mystring).lower().replace("--","-")
mystring

'the-legend-of-zelda-breath-of-the-wild'

In [ ]:
# Grabbing the Metascore

game_scores = {}
problem_games = []
counter = 1

# zip the two 
zipped_list =list(zip(games_df["full_platform_name"].tolist(),games_df["Name"].tolist()))

for platform, name in zipped_list:
    x = re.compile("\W|\s")
    edited_game_title= re.sub(x,"-", name).lower().replace("--","-")
    # periods at the end of the url will show up as a dash. This conditional is to get rid of it.
    if edited_game_title[-1] == "-":
        edited_game_title = edited_game_title[:-1]
        print(edited_game_title)
    else:
        print(edited_game_title)
    browser.visit(f"https://www.metacritic.com/game/{platform}/{edited_game_title}")
    # fill in my search term to the Metacritic Browser and us "\n" to press "enter" and initialize search
    
    html= browser.html
    my_soup = soup(html, "lxml")
    try:
        metascore = my_soup.find("a", class_="metascore_anchor").get_text()
        game_scores[game]= metascore
        print("SUCCESSFUL")
        print(counter)
        counter+=1
    except:
        problem_games.append(game)
        print("THIS IS A PROBLEM CHILD")
        print(counter)
        counter+=1

wii-sports
SUCCESSFUL
1
mario-kart-wii
SUCCESSFUL
2
wii-sports-resort
SUCCESSFUL
3
new-super-mario-bros
SUCCESSFUL
4
wii-play
SUCCESSFUL
5
new-super-mario-bros-wii
SUCCESSFUL
6
nintendogs
THIS IS A PROBLEM CHILD
7
mario-kart-ds
SUCCESSFUL
8
wii-fit
SUCCESSFUL
9
wii-fit-plus
SUCCESSFUL
10
kinect-adventures
SUCCESSFUL
11
grand-theft-auto-v
SUCCESSFUL
12
grand-theft-auto-san-andreas
SUCCESSFUL
13
brain-age-train-your-brain-in-minutes-a-day
SUCCESSFUL
14
pokemon-diamond-pokemon-pearl
THIS IS A PROBLEM CHILD
15
grand-theft-auto-v
SUCCESSFUL
16
grand-theft-auto-vice-city
SUCCESSFUL
17
pokemon-ruby-pokemon-sapphire
THIS IS A PROBLEM CHILD
18
pokemon-black-pokemon-white
THIS IS A PROBLEM CHILD
19
brain-age-2-more-training-in-minutes-a-day
SUCCESSFUL
20
gran-turismo-3-a-spec
SUCCESSFUL
21
call-of-duty-modern-warfare-3
SUCCESSFUL
22
call-of-duty-black-ops
SUCCESSFUL
23
call-of-duty-black-ops-3
THIS IS A PROBLEM CHILD
24
call-of-duty-black-ops-ii
SUCCESSFUL
25
call-of-duty-black-ops-ii
SUCCESSFUL

In [153]:
names = games_df["Name"].tolist()
x = re.compile("\W|\s")
for name in names:
    new_name = re.sub(x,"-", name).lower().replace("--","-")
    if new_name[-1] == "-":
        new_name = new_name[:-1]
        print(final_name)
    else:
        print(new_name)

wii-sports
mahou-sensei-negima--neo-pactio-fight
mario-kart-wii
wii-sports-resort
pokemon-red-pokemon-blue
tetris
mahou-sensei-negima--neo-pactio-fight
wii-play
new-super-mario-bros-wii
duck-hunt
nintendogs
mario-kart-ds
pokemon-gold-pokemon-silver
wii-fit
wii-fit-plus
mahou-sensei-negima--neo-pactio-fight
grand-theft-auto-v
grand-theft-auto-san-andreas
super-mario-world
brain-age-train-your-brain-in-minutes-a-day
pokemon-diamond-pokemon-pearl
super-mario-land
super-mario-bros-3
grand-theft-auto-v
grand-theft-auto-vice-city
pokemon-ruby-pokemon-sapphire
pokemon-black-pokemon-white
brain-age-2-more-training-in-minutes-a-day
gran-turismo-3-a-spec
call-of-duty-modern-warfare-3
pokémon-yellow-special-pikachu-edition
call-of-duty-black-ops
pokemon-x-pokemon-y
call-of-duty-black-ops-3
call-of-duty-black-ops-ii
call-of-duty-black-ops-ii
call-of-duty-modern-warfare-2
call-of-duty-modern-warfare-3
grand-theft-auto-iii
super-smash-bros-brawl
call-of-duty-black-ops
animal-crossing-wild-world
mari

rocksmith-2014
dead-space-3
wheel-of-fortune
disney-infinity
dark-souls
puzzler-collection
jampack-winter-2000
dead-space-3
phoenix-wright-ace-attorney
nfl-fever-2002
the-simpsons-hit--run
littlest-pet-shop-winter
lego-harry-potter-years-5-7
tom-clancy-s-splinter-cell-blacklist
ace-combat-x-skies-of-deception
ncaa-football-12
lego-pirates-of-the-caribbean-the-video-game
band-hero
cars-race-o-rama
oddworld-abe-s-exoddus
eyepet
scooby-doo-first-frights
midnight-club-la-remix
the-bouncer
guitar-hero-aerosmith
wwe-smackdown-vs-raw-2008
fifa-soccer-10
shadow-the-hedgehog
farming-simulator-2013
fallout-3
cars
pro-evolution-soccer-2008
far-cry-3
xenoblade-chronicles
just-dance-2014
mario-party-advance
sonic--sega-all-stars-racing
tetris-2
nascar-99
air-sea-battle
suikoden-iii
kingdom-hearts-re-coded
south-park-the-stick-of-truth
driver-san-francisco
sniper-elite-v2
far-cry-4
grand-theft-auto-san-andreas
tom-clancy-s-splinter-cell-blacklist
pro-evolution-soccer-2014
madden-nfl-16
madden-nfl-10

tony-hawk-s-project-8
yu-gi-oh-world-championship-tournament-2004
cabela-s-big-game-hunter-2005-adventures
hotel-dusk-room-215
knockout-kings-2001
army-men-sarge-s-heroes
phineas-and-ferb-across-the-2nd-dimension
petz-dogz-2
battlefield-2-modern-combat
wolfenstein
all-star-baseball-2002
backyard-nfl-football
need-for-speed-most-wanted
nascar-rumble
tony-hawk-shred
tony-hawk-s-proving-ground
wwe-raw-2
resident-evil-2
mega-man-x2
manhunt-2
xenoblade-chronicles
fullmetal-alchemist-and-the-broken-angel
monster-high-ghoul-spirit
final-fantasy-iv-the-complete-collection
frontlines-fuel-of-war
lips-number-one-hits
mahou-sensei-negima--neo-pactio-fight
batman-begins
viva-pinata-pocket-paradise
dr-seuss-the-cat-in-the-hat
barbie-groovy-games
yu-gi-oh-reshef-of-destruction
blinx-the-time-sweeper
madagascar-escape-2-africa
archer-maclean-s-mercury
espn-nfl-football
fatal-fury-special
catz
major-league-baseball-2k5
ds-bimoji-training
wwe-2k15
petz-catz-clan
soul-sacrifice
g-force
tiger-woods-pga-t

mlb-06-the-show
f1-race-stars
over-the-hedge
naruto-shippuden-ultimate-ninja-impact
just-dance-2016
charlie-and-the-chocolate-factory
pirates-of-the-caribbean-at-world-s-end
uefa-euro-2008-austria-switzerland
kurushi-final-mental-blocks
style-lab-jewelry-design
risk--battleship--clue
sd-gundam-g-generation-world
lego-pirates-of-the-caribbean-the-video-game
the-wild-thornberrys-movie
mahou-sensei-negima--neo-pactio-fight
dynasty-warriors-5-empires
soulcalibur
wipeout-3
etrian-odyssey
conan
classic-nes-series-ice-climber
skylanders-trap-team
avatar-the-last-airbender--the-burning-earth
mahou-sensei-negima--neo-pactio-fight
tenchu-z
freeway
driver-2-advance
monster-rancher-4
iron-man-2
chicken-riot
tim-burton-s-the-nightmare-before-christmas-oogie-s-revenge
iron-storm
harry-potter-and-the-deathly-hallows--part-2
motorstorm-arctic-edge
the-fairly-odd-parents-shadow-showdown
radiant-historia
marvel-super-heroes
super-bust-a-move
hitman-contracts
thunderstrike-operation-phoenix
alien-s-retur

rocket-league
wipeout-2
the-idolm-ster-sp-wandering-star--perfect-sun--missing-moon
critical-depth
barbie-as-the-island-princess
petz-rescue-wildlife-vet
mahou-sensei-negima--neo-pactio-fight
saka-tsuku-ds-touch-and-direct
super-swing-golf
the-italian-job
mega-man-battle-network-5-team-colonel--protoman
barbie-and-the-three-musketeers
star-ocean
freedom-fighters
naruto-ninja-council-2-european-version
soldier-of-fortune-ii-double-helix
pro-evolution-soccer-2011
disney-princess-enchanted-journey
final-fantasy-xi-treasures-of-aht-urhgan
alundra
world-soccer-winning-eleven-2010-aoki-samurai-no-chousen
bratz-girlz-really-rock
super-r-type
rampage-2-universal-tour
super-robot-taisen-f-kanketsuhen
tiger-woods-pga-tour-2003
blowout
nfl-blitz-pro
mysims-party
tron-evolution--battle-grids
the-raiden-project
hooked-real-motion-fishing
pocket-fighter
bicycle-casino-2005
valhalla-knights-2
-hack-g-u-vol-2-reminisce
disgaea-afternoon-of-darkness
ratchet--clank-full-frontal-assault
ncaa-football-09


mtv-music-generator-3-this-is-the-remix
cake-mania-3
-hack-link
terminator-3-rise-of-the-machines
star-wars-knights-of-the-old-republic
ssx-on-tour
ncaa-college-basketball-2k3
class-of-heroes
yogi-bear-the-video-game
mahou-sensei-negima--neo-pactio-fight
the-legend-of-heroes-ii-prophecy-of-the-moonlight-witch
cranium-kabookii
hero-s-saga-laevatein-tactics
doom
gundam-breaker-2
trinity-souls-of-zill-o-ll
mary-kate-and-ashley-winners-circle
parappa-the-rapper-2
hexyz-force
outrun-2
okami
marvel-vs-capcom-2-new-age-of-heroes
hitman-blood-money
soukaigi
naruto-shippuden-ninja-destiny-3
mahou-sensei-negima--neo-pactio-fight
pipe-mania
gretzky-nhl-06
shonen-jump-s-shaman-king-master-of-spirits
pangya-fantasy-golf
nfl-blitz-20-03
sumikko-gurashi-omise-hajimerundesu
pirates-the-legend-of-black-kat
fur-fighters-viggo-s-revenge
krull
yu-gi-oh-monster-capture-gb
risen
top-spin-3
mahou-sensei-negima--neo-pactio-fight
the-da-vinci-code
watch-dogs
mr-driller-drill-spirits
arctic-tale
scooby-doo-nigh

roary-the-racing-car
ski-and-shoot
mugen-keitai-disgaea
scrabble
samurai-warriors-4-empires
action-man-operation-extreme
blazblue-continuum-shift-extend
tornado-outbreak
mystic-heroes
pachitte-chonmage-tatsujin-10-pachinko-fuyu-no-sonata
defender
the-sims-2-apartment-life
mlb-superstars
captain-tsubasa-aratanaru-densetsu-joshou
remington-super-slam-hunting-alaska
mahou-sensei-negima--neo-pactio-fight
clockwork-knight
motogp-14
freestyle-metal-x
mahou-sensei-negima--neo-pactio-fight
sidewinder-max
the-1-pyramid
burnout-paradise-the-ultimate-box
kenka-banchou-2-full-throttle
zoids-assault
pro-yaky-spirits-2014
dark-angel-vampire-apocalypse
the-mummy-tomb-of-the-dragon-emperor
spider-man-edge-of-time
momu-chan-diet-wii-figurobics-by-chon-dayon
karaoke-revolution-presents-american-idol-encore-2
atelier-totori-the-adventurer-of-arland
mad-dog-mccree-gunslinger-pack
no-more-heroes-red-zone
mahou-sensei-negima--neo-pactio-fight
bakugan-rise-of-the-resistance
thomas--friends-hero-of-the-rails


diva-girls-divas-on-ice
mahou-sensei-negima--neo-pactio-fight
a-train-6
meitantei-conan-tantei-ryoku-trainer
motogp
espn-international-winter-sports-2002
mahou-sensei-negima--neo-pactio-fight
mahou-sensei-negima--neo-pactio-fight
gold-and-glory-the-road-to-el-dorado
nba-live-07
choro-q-marine-q-boat
gunpey
battlebots-beyond-the-battlebox
mx-vs-atv-supercross-encore
yuugen-gaisha-brave-company
mighty-beanz-pocket-puzzles
ken-to-mahou-to-gakuen-mono-3d
history-great-empires-rome
the-king-of-fighters-96
friends-the-one-with-all-the-trivia
mahou-sensei-negima--neo-pactio-fight
backyard-nfl-football-08
coraline
gunbird
imagine-party-planner
dream-c-club-portable
tom-clancy-s-rainbow-six-3
elemental-gearbolt
space-chimps
tony-hawk-s-pro-skater-5
wwf-wrestlemania-the-arcade-game
handball-16
cabela-s-big-game-hunter-2005-adventures
grand-kingdom
mega-man-8-anniversary-collector-s-edition
v-rally-3
nobunaga-no-yabou-souzou
whiplash
grid
pro-yakyuu-famista-2011
impact-racing
my-horse--me-riding-

kung-fu-panda-legendary-warriors
mahjong-kakutou-club-zenkoku-taisenban
hayate-no-gotoku-nightmare-paradise
nba-08
secret-files-2-puritas-cordis
nanashi-no-game-me
heroes-of-hellas-2-olympia
young-justice-legacy
syberia
bladestorm-nightmare
natgeo-challenge-wild-life
higurashi-no-naku-koro-ni-iki
happinesscharge-precure-kawarun-collection
battle-vs-chess
dream-pinball-3d
dice-dna-integrated-cybernetic-enterprises
battlefield-1942-the-ww-ii-anthology
treasure-gaust-gaust-diver-crimson-red--deep-purple
mahou-sensei-negima--neo-pactio-fight
mahou-sensei-negima--neo-pactio-fight
wallace--gromit-in-project-zoo
rugby-league-live
grid-runner
saihai-no-yukue
metal-dungeon
drome-racers
guitar-freaks-v--drummania-v
tago-akira-no-atama-no-taisou-dai-1-shuu-nazotoki-sekai-isshuu-ryokou
driver-parallel-lines
doom
gekido-advance-kintaro-s-revenge
mahou-sensei-negima--neo-pactio-fight
girls-only
future-tactics-the-uprising
power-poke-dash
hisshou-pachinko-pachi-slot-kouryaku-series-ds-vol-4-shinseiki

farming-2017--the-simulation
grisaia-no-kajitsu-la-fruit-de-la-grisaia
scarlett-nichijou-no-kyoukaisen
mini-desktop-racing
yattaman-wii-bikkuridokkiri-machine-de-mou-race-da-koron
neo-angelique-special
rugby-challenge-3
damnation
outdoors-unleashed-africa-3d
pga-european-tour
real-rode
fit--fun
planet-monsters
carmageddon-64
bust-a-move-3000
breach
secret-files-2-puritas-cordis
mezase--tsuri-master-ds
mega-brain-boost
chou-ezaru-wa-akai-hana-koi-wa-tsuki-ni-shirube-kareru
eiyuu-densetsu-sora-no-kiseki-material-collection-portable
myst-iv-revelation
plushees
woody-woodpecker-in-crazy-castle-5
men-in-black-ii-alien-escape
score-international-baja-1000-the-official-game
know-how-2
spirits--spells


['Wii Sports',
 'Super Mario Bros.',
 'Mario Kart Wii',
 'Wii Sports Resort',
 'Pokemon Red/Pokemon Blue',
 'Tetris',
 'New Super Mario Bros.',
 'Wii Play',
 'New Super Mario Bros. Wii',
 'Duck Hunt',
 'Nintendogs',
 'Mario Kart DS',
 'Pokemon Gold/Pokemon Silver',
 'Wii Fit',
 'Wii Fit Plus',
 'Kinect Adventures!',
 'Grand Theft Auto V',
 'Grand Theft Auto: San Andreas',
 'Super Mario World',
 'Brain Age: Train Your Brain in Minutes a Day',
 'Pokemon Diamond/Pokemon Pearl',
 'Super Mario Land',
 'Super Mario Bros. 3',
 'Grand Theft Auto V',
 'Grand Theft Auto: Vice City',
 'Pokemon Ruby/Pokemon Sapphire',
 'Pokemon Black/Pokemon White',
 'Brain Age 2: More Training in Minutes a Day',
 'Gran Turismo 3: A-Spec',
 'Call of Duty: Modern Warfare 3',
 'Pokémon Yellow: Special Pikachu Edition',
 'Call of Duty: Black Ops',
 'Pokemon X/Pokemon Y',
 'Call of Duty: Black Ops 3',
 'Call of Duty: Black Ops II',
 'Call of Duty: Black Ops II',
 'Call of Duty: Modern Warfare 2',
 'Call of Duty: Moder